<a href="https://colab.research.google.com/github/BrunoMachadoF/Titanic-Kaggle-Eda-Studies/blob/main/Titanic_Studies.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# 1) Introdução

**Objetivo.** Conduzir uma EDA do Titanic para o plano de estudos (Semana 1: Python Fundamentals, Estatística Descritiva, Git), explorando fatores associados à sobrevivência e documentando achados com testes estatísticos e gráficos interativos.

**Escopo.**

* Dataset: `train.csv` (com `Survived`) e `test.csv` (sem rótulo).
* Saídas: gráficos Plotly, tabelas e interpretações

# 2) Reprodutibilidade e Configuração

In [79]:
# ------------------------
# Imports
# ------------------------
from __future__ import annotations

import os
import re
import logging
import numpy as np
import pandas as pd
import plotly.express as px

from dataclasses import dataclass
from typing import Tuple, Dict, Any, Iterable
from scipy import stats

**Descrição**

* Ativa a avaliação postergada de anotações (`from __future__ import annotations`), permitindo usar type hints com referências futuras e reduzindo custo de importação.
* Módulos padrão: `os` (arquivos/paths), `re` (regex), `logging` (logs estruturados), `dataclass` (configurações imutáveis/claras), `typing` (tipagem estática).
* Stack científico: `numpy` (vetorização), `pandas` (ETL/EDA tabular), `scipy.stats` (qui-quadrado, Mann-Whitney, Kruskal, correlações), `plotly.express` (gráficos interativos).

In [80]:
# ------------------------
# Configurações globais
# ------------------------
@dataclass
class Config:
    outdir: str = "reports"
    seed: int = 42
    min_freq_cat: int = 10     # frequência mínima p/ colapsar categorias raras
    save_figs: bool = True     # salvar HTML das figuras Plotly

CFG = Config()

# Tema/tamanho padrão dos gráficos Plotly
px.defaults.template = "plotly_white"
px.defaults.width = 900
px.defaults.height = 500

**Descrição**

* Define uma dataclass de configuração para concentrar parâmetros do projeto: pasta de saída, semente de aleatoriedade, limiar para colapsar categorias raras e flag para salvar gráficos. Facilita reprodutibilidade e evita “números mágicos” espalhados.
* Instancia um objeto de configuração único (CFG) que será referenciado nas demais funções, mantendo o código mais limpo e consistente.
* Padroniza o tema e dimensões do Plotly, garantindo identidade visual e evitando repetição de argumentos em cada figura.



In [81]:
# ------------------------
# Utilitários de ambiente
# ------------------------
def ensure_outdir(path: str) -> None:
    os.makedirs(path, exist_ok=True)

def setup_logging(level: int = logging.INFO) -> None:
    logging.basicConfig(level=level, format="%(asctime)s | %(levelname)s | %(message)s")

def save_fig(fig, name: str, outdir: str = CFG.outdir) -> None:
    """Salva figura em HTML com nome normalizado."""
    if not CFG.save_figs:
        return
    ensure_outdir(outdir)
    safe = re.sub(r"\s+", "_", name.strip().lower())
    fig.write_html(os.path.join(outdir, f"{safe}.html"))

def collapse_rare(df: pd.DataFrame, col: str, min_freq: int, new_label: str = "Rare") -> pd.DataFrame:
    """Agrupa categorias com frequência < min_freq em 'new_label'."""
    vc = df[col].value_counts(dropna=False)
    rare = vc[vc < min_freq].index
    df[col] = df[col].where(~df[col].isin(rare), new_label)
    return df

**Descrição**

* **ensure\_outdir**

  Garante que o diretório de saída exista antes de gravar arquivos. Evita erros de “No such file or directory” ao salvar relatórios e figuras.

* **setup\_logging**

  Configura logging com nível e formato padronizados, permitindo diagnóstico consistente ao longo do notebook e facilitando debug e auditoria.

* **save\_fig**

  Normaliza o nome do arquivo (troca espaços por sublinhado, converte para minúsculas), cria o diretório se necessário e salva a figura Plotly em HTML. Usa a flag global para ativar/desativar salvamento, útil para rodadas rápidas.

* **collapse\_rare**

  Agrupa categorias com frequência abaixo do limiar em um rótulo único (“Rare”), reduzindo sparsidade e risco de overfitting em análises e modelos. Preserva contagem de NaN (não os substitui), o que é desejável em EDA.

In [82]:
# ------------------------
# Helpers estatísticos
# ------------------------
def chi_square_from_crosstab(tab: pd.DataFrame) -> Dict[str, Any]:
    chi2, p, dof, exp = stats.chi2_contingency(tab.values)
    n = tab.values.sum()
    v = np.sqrt(chi2 / (n * (min(tab.shape) - 1)))
    return {"chi2": chi2, "p": p, "dof": dof, "cramers_v": v, "expected_min": float(exp.min())}

def mannwhitney_stats(x: Iterable[float], y: Iterable[float]) -> Dict[str, Any]:
    x = np.asarray(list(x), dtype=float)
    y = np.asarray(list(y), dtype=float)
    U, p = stats.mannwhitneyu(x, y, alternative="two-sided")
    n1, n0 = len(x), len(y)
    rank_biserial = 1 - 2 * U / (n1 * n0)
    auc = U / (n1 * n0)
    return {"U": float(U), "p": float(p), "rank_biserial": float(rank_biserial), "auc": float(auc), "n1": n1, "n0": n0}

**Descrição**

* **chi\_square\_from\_crosstab**

  Executa qui-quadrado de independência a partir de uma tabela de contingência (pandas crosstab), retornando χ², p-valor, graus de liberdade, Cramér’s V e o menor valor esperado (para checar a premissa de células esperadas). Útil para medir associação entre variáveis categóricas e quantificar a força do efeito com Cramér’s V.

* **mannwhitney\_stats**

  Roda o teste de Mann-Whitney U (bicaudal) entre dois grupos numéricos, retornando U, p-valor, além de dois tamanhos de efeito: AUC (U/(n1·n0)) e rank-biserial. Fornece também as amostragens n1 e n0 para transparência.

In [83]:
# ------------------------
# Inicialização do ambiente
# ------------------------
setup_logging()
np.random.seed(CFG.seed)


**Descrição**

* Ativa o logging padronizado para toda a execução, permitindo acompanhar etapas e mensagens com timestamp e nível de severidade.
* Fixa a semente do gerador pseudoaleatório do NumPy para resultados reproduzíveis em operações que dependem de aleatoriedade.



# 3) Carregamento e Preparação dos Dados

In [84]:
# ---------------------------------
# Caminhos
# ---------------------------------
TRAIN_PATH = "https://github.com/BrunoMachadoF/titanic-kaggle-competition/raw/main/data/train.csv"
TEST_PATH  = "https://github.com/BrunoMachadoF/titanic-kaggle-competition/raw/main/data/test.csv"

**Descrição**

* Define caminhos centralizados para os datasets de treino e teste usando URLs “raw” do GitHub. Isso documenta a origem dos dados e permite carregar diretamente via pandas, mantendo o fluxo reprodutível.
* Mantém ausente qualquer referência a gender\_submission, evitando vazamento de rótulos no processo de EDA e modelagem.


In [85]:
# ---------------------------------
# Funções de ingestão e preparação
# ---------------------------------
def load_data(train_path: str, test_path: str) -> Tuple[pd.DataFrame, pd.DataFrame]:
    """
    Lê os CSVs de train e test. Garante que test não contenha 'Survived'.
    """
    train = pd.read_csv(train_path)
    test = pd.read_csv(test_path)
    if "Survived" in test.columns:
        test = test.drop(columns="Survived")
    return train, test

def make_complete(train: pd.DataFrame, test: pd.DataFrame) -> Tuple[pd.DataFrame, pd.DataFrame, pd.DataFrame]:
    """
    Concatena train e test em complete_df com 'Survived' = <NA> no test.
    Retorna (complete_df, train_only, test_only).
    """
    train2 = train.copy()
    train2["Survived"] = train2["Survived"].astype("Int64")

    test2 = test.copy()
    test2["Survived"] = pd.Series([pd.NA] * len(test2), dtype="Int64")

    complete = pd.concat([train2, test2], ignore_index=True, sort=False)
    train_only = complete[complete["Survived"].notna()].copy()
    test_only = complete[complete["Survived"].isna()].drop(columns="Survived").copy()
    return complete, train_only, test_only

def add_family_features(df: pd.DataFrame) -> pd.DataFrame:
    """
    Cria FamilySize (= SibSp + Parch + 1) e IsAlone (1 se FamilySize==1).
    """
    out = df.copy()
    out["SibSp"] = out["SibSp"].fillna(0)
    out["Parch"] = out["Parch"].fillna(0)
    out["FamilySize"] = out["SibSp"] + out["Parch"] + 1
    out["IsAlone"] = (out["FamilySize"] == 1).astype(int)
    return out

def add_cabin_features(df: pd.DataFrame) -> pd.DataFrame:
    """
    Cria HasCabin (1=tem Cabin não nulo) e Deck (1ª letra de Cabin, 'Unknown' quando ausente).
    """
    out = df.copy()
    out["HasCabin"] = (~out["Cabin"].isna()).astype(int)
    out["Deck"] = out["Cabin"].astype(str).str[0]
    out.loc[out["Cabin"].isna(), "Deck"] = "Unknown"
    return out

def extract_title(df: pd.DataFrame, name_col: str = "Name") -> pd.DataFrame:
    """
    Extrai Title de Name (trecho entre vírgula e ponto), normaliza e colapsa categorias raras.
    """
    out = df.copy()
    out["Title"] = out[name_col].str.extract(r",\s*([^\.]+)\.", expand=False).str.strip()

    normalize = {
        "Mlle": "Miss", "Ms": "Miss", "Mme": "Mrs",
        "Lady": "Lady", "the Countess": "Lady",
        "Capt": "Officer", "Col": "Officer", "Major": "Officer", "Dr": "Officer", "Rev": "Officer",
        "Don": "Noble", "Sir": "Noble", "Jonkheer": "Noble", "Dona": "Noble",
    }
    out["Title"] = out["Title"].replace(normalize)
    out = collapse_rare(out, "Title", min_freq=CFG.min_freq_cat, new_label="Rare")
    return out

**Descrição**

* **load\_data**

  Lê os CSVs de treino e teste e garante que o teste não contenha a coluna Survived. Evita contaminação do alvo já na origem.

* **make\_complete**

  Concatena treino e teste em um único DataFrame: em treino, Survived vira Int64; em teste, Survived é definido como <NA>. Retorna também subconjuntos train\_only e test\_only para análises com e sem alvo, prevenindo vazamento.

* **add\_family\_features**

  Cria FamilySize = SibSp + Parch + 1 e o indicador IsAlone (1 quando FamilySize == 1). Essas variáveis capturam estrutura familiar, útil para explicar sobrevivência.

* **add\_cabin\_features**

  Gera HasCabin (1 se há informação de cabine) e extrai Deck como a primeira letra de Cabin; casos ausentes recebem o rótulo “Unknown”. Essas features funcionam como proxies de localização/classe.

* **extract\_title**

  Extrai Title do Name via regex (trecho entre vírgula e ponto), normaliza títulos equivalentes e colapsa categorias raras conforme min\_freq\_cat. O título sintetiza sexo, idade e status social, aumentando poder explicativo.

In [86]:
# ---------------------------------
# Execução de preparo
# ---------------------------------

# 1) Carrega dados
train_df, test_df = load_data(TRAIN_PATH, TEST_PATH)

# 2) Cria complete_df seguro
complete_df, train_only, test_only = make_complete(train_df, test_df)

# 3) Engenharia de atributos básica (aplicada no complete_df)
complete_df = add_family_features(complete_df)
complete_df = add_cabin_features(complete_df)
complete_df = extract_title(complete_df)

# 4) Atualiza os subconjuntos com as novas colunas
train_only = complete_df[complete_df["Survived"].notna()].copy()
test_only  = complete_df[complete_df["Survived"].isna()].drop(columns="Survived").copy()

**Descrição**

* Carrega os arquivos de treino e teste a partir das URLs definidas, mantendo o teste sem a coluna Survived para evitar vazamento do alvo.
* Constrói um complete\_df “seguro”, onde o treino mantém Survived e o teste recebe Survived como <NA>; em seguida, separa train\_only e test\_only para análises com e sem alvo.
* Aplica a engenharia de atributos no complete\_df: FamilySize/IsAlone (estrutura familiar), HasCabin/Deck (informação de cabine) e Title (extraído de Name), ampliando o poder descritivo e interpretativo.
* Recria train\_only e test\_only após a engenharia para garantir que os subconjuntos possuam as novas colunas derivadas, evitando inconsistências em análises subsequentes.

# 4) Dicionário de Dados & Qualidade

In [87]:
# -------------------------
# Dataset alvo desta seção
# -------------------------
df_all   = complete_df        # train + test (Survived=<NA> no test)
df_train = train_only         # apenas registros com Survived conhecido (sem vazamento)

# -------------------------
# 4.1 Dicionário de Dados
# -------------------------
schema = pd.DataFrame({
    "coluna": df_all.columns,
    "dtype":  df_all.dtypes.astype(str).values,
    "n_nulos": df_all.isna().sum().values,
    "n_nao_nulos": df_all.notna().sum().values,
    "n_unicos": df_all.nunique(dropna=True).values,
})
schema["pct_nulos"] = (schema["n_nulos"] / len(df_all)).round(3)
# Exemplo de valor (primeiro não-nulo)
schema["exemplo"] = [
    (df_all[c].dropna().iloc[0] if df_all[c].notna().any() else np.nan) for c in df_all.columns
]
schema = schema.sort_values("coluna", ignore_index=True)
display(schema)

# (opcional) salvar
ensure_outdir(CFG.outdir)
schema.to_csv(f"{CFG.outdir}/dic_dados_schema.csv", index=False)

,coluna,dtype,n_nulos,n_nao_nulos,n_unicos,pct_nulos,exemplo
0,Age,float64,263,1046,98,0.201,22.0
1,Cabin,object,1014,295,186,0.775,C85
2,Deck,object,0,1309,9,0.000,Unknown
3,Embarked,object,2,1307,3,0.002,S
4,FamilySize,int64,0,1309,9,0.000,2
5,Fare,float64,1,1308,281,0.001,7.25
6,HasCabin,int64,0,1309,2,0.000,0
7,IsAlone,int64,0,1309,2,0.000,0
8,Name,object,0,1309,1307,0.000,"Braund, Mr. Owen Harris"
9,Parch,int64,0,1309,8,0.000,0


**Descrição**

* df\_all referencia a união de treino e teste e é usado para inspeções que não dependem do alvo (schema, faltantes, descrições gerais), já que no teste Survived está como ausente.
* df\_train isola apenas as linhas com Survived conhecido, garantindo que qualquer análise ou teste estatístico que envolva o alvo não incorra em vazamento.
* Constrói uma tabela-resumo com nome da coluna, tipo (`dtype`), contagem de nulos e não nulos, número de valores únicos e a fração de nulos (`pct_nulos`), útil para mapear o schema e priorizar limpeza.
* Adiciona um exemplo por coluna (primeiro valor não nulo) para contextualizar o conteúdo de cada campo.
* Ordena alfabeticamente e exibe (`display`); opcionalmente salva em CSV em `reports/`, garantindo rastreabilidade.


In [88]:
# -------------------------
# 4.2 Qualidade de Dados
# -------------------------
# Faltantes por coluna (barra)
miss = (df_all.isna().sum().sort_values(ascending=False)
        .rename("n_nulos").to_frame().reset_index().rename(columns={"index":"coluna"}))
fig_miss = px.bar(
    miss, x="coluna", y="n_nulos",
    title="Valores faltantes por coluna",
    labels={"coluna":"Coluna", "n_nulos":"Qtde de nulos"}
)
fig_miss.update_layout(xaxis={'categoryorder':'total descending'})
fig_miss.show()
save_fig(fig_miss, "faltantes_por_coluna")

# Duplicatas por PassengerId (integridade de chave)
dup_pid = int(df_all.duplicated(subset=["PassengerId"]).sum()) if "PassengerId" in df_all.columns else 0
print(f"Duplicatas por PassengerId: {dup_pid}")

# Regras de consistência simples
checks = {}
if "Pclass" in df_all.columns:
    checks["pclass_invalida"] = int(~df_all["Pclass"].isin([1,2,3]).sum())
if "Age" in df_all.columns:
    checks["age_negativa"] = int((df_all["Age"] < 0).sum())
    checks["age_maior_100"] = int((df_all["Age"] > 100).sum())
if "Fare" in df_all.columns:
    checks["fare_negativa"] = int((df_all["Fare"] < 0).sum())

print("Checagens de consistência:", checks)

Duplicatas por PassengerId: 0
Checagens de consistência: {'pclass_invalida': -1310, 'age_negativa': 0, 'age_maior_100': 0, 'fare_negativa': 0}


**Interpretação**

* **Ausências críticas:**

Age com 20,1% de nulos (263) e Cabin com 77,5% (1014). Age requer imputação estratificada (ex.: Sex × Pclass × Title). Cabin é inviável como bruta; use derivadas HasCabin e Deck.

* **Alvo:**

 Survived está ausente em 31,9% (418), consistente com o conjunto de teste; dtype Int64 permite NAs sem quebrar operações.

* **Chave:**

PassengerId é único (1309/1309), adequado como identificador.

* **Alta cardinalidade:**

Name (1307 únicos) e Ticket (929) não são úteis diretamente; extrair Title (já feito) e, se necessário, prefixos de Ticket.

* **Categóricas principais:**

Sex (2), Embarked (3), Pclass (3). Tratar como category; Pclass é ordinal (1 < 2 < 3), evitar interpretar como contínua.

* **Derivadas coesas:**

FamilySize (9 valores), IsAlone e HasCabin binárias; Deck possui 9 categorias com “Unknown” dominante (originada da falta em Cabin).

* **Fare:**

apenas 1 nulo; imputar por mediana estratificada (ex.: Pclass × Embarked). Distribuição com muitos valores únicos (281) e cauda longa.

* **Consistência geral:**

Sem duplicatas reportadas nesta visão; schema e percentuais de nulos condizem com o Titanic “padrão”.

**Implicações para a EDA/modelagem**

* Priorizar imputações de Age/Fare/Embarked; usar HasCabin/Deck/Title em vez de Cabin/Name.
* Em testes de associação com Deck, verificar contagens esperadas devido a categorias raras; agrupar se necessário.

In [89]:
# -------------------------
# 4.3 Resumos estatísticos
# -------------------------
# Numéricos (df_all)
desc_num_all = df_all.select_dtypes(include=[np.number]).describe().T
display(desc_num_all)

# Categóricos (df_all)
desc_cat_all = df_all.select_dtypes(include=['object','category']).describe().T
display(desc_cat_all)

# Alvo (df_train) — distribuição e taxa de sobrevivência
if "Survived" in df_train.columns:
    tgt_counts = df_train["Survived"].value_counts(dropna=False).rename_axis("Survived").to_frame("contagem")
    tgt_counts["proporcao"] = (tgt_counts["contagem"] / tgt_counts["contagem"].sum()).round(3)
    display(tgt_counts)
    print(f"Taxa de sobrevivência (train): {df_train['Survived'].mean():.3f}")

# (opcional) salvar resumos
ensure_outdir(CFG.outdir)
desc_num_all.to_csv(f"{CFG.outdir}/describe_numericos.csv")
desc_cat_all.to_csv(f"{CFG.outdir}/describe_categoricos.csv")
if "Survived" in df_train.columns:
    tgt_counts.to_csv(f"{CFG.outdir}/alvo_distribuicao_train.csv")

,count,mean,std,min,25%,50%,75%,max
PassengerId,1309.0,655.0,378.020061,1.0,328.0,655.0,982.0,1309.0
Survived,891.0,0.383838,0.486592,0.0,0.0,0.0,1.0,1.0
Pclass,1309.0,2.294882,0.837836,1.0,2.0,3.0,3.0,3.0
Age,1046.0,29.881138,14.413493,0.17,21.0,28.0,39.0,80.0
SibSp,1309.0,0.498854,1.041658,0.0,0.0,0.0,1.0,8.0
Parch,1309.0,0.385027,0.86556,0.0,0.0,0.0,0.0,9.0
Fare,1308.0,33.295479,51.758668,0.0,7.8958,14.4542,31.275,512.3292
FamilySize,1309.0,1.883881,1.583639,1.0,1.0,1.0,2.0,11.0
IsAlone,1309.0,0.603514,0.489354,0.0,0.0,1.0,1.0,1.0
HasCabin,1309.0,0.225363,0.417981,0.0,0.0,0.0,0.0,1.0


,count,unique,top,freq
Name,1309,1307,"Kelly, Mr. James",2
Sex,1309,2,male,843
Ticket,1309,929,CA. 2343,11
Cabin,295,186,C23 C25 C27,6
Embarked,1307,3,S,914
Deck,1309,9,Unknown,1014
Title,1309,6,Mr,757


,contagem,proporcao
Survived,,
0,549,0.616
1,342,0.384


Taxa de sobrevivência (train): 0.384


**Interpretação**

* **Números (df\_all)**

  Gera estatísticas descritivas para todas as colunas numéricas (count, média, desvio-padrão, quartis, min/máx). Útil para entender escala, dispersão e possíveis outliers antes das comparações bivariadas.

* **Categóricas (df\_all)**

  Calcula contagem, número de categorias, categoria mais frequente e sua frequência. Ajuda a identificar desbalanceamentos e categorias raras.

* **Alvo (df\_train)**

  Mostra a distribuição de Survived no treino (contagens e proporção) e a taxa média de sobrevivência. Evita vazamento ao não usar linhas do conjunto de teste.

* **Persistência**

  Salva os resumos em CSV dentro de reports/ para auditoria e reuso em relatórios externos.



# 5) Estatística Descritiva

**Tabela 1 — Numéricas (df\_all)**

* Survived aparece com count=891 (apenas o trecho de treino tem rótulo), média ≈ 0,384; isso é esperado no complete\_df e não indica vazamento.
* Pclass tem mediana 3 e média ≈ 2,29, sugerindo predominância de 3ª classe.
* Age com count=1046 indica \~20% faltantes; média ≈ 29,9 e mediana 28 reforçam concentração em adultos jovens; máx 80 é plausível.
* Fare tem cauda pesada (máx ≈ 512,33; mediana 14,45), com 1 valor faltante e mínimos em 0 que podem indicar passagens gratuitas/registradas como zero.
* Estrutura familiar: SibSp e Parch muito concentrados em 0; FamilySize mediana 1 e máx 11, coerente com poucos grupos grandes; IsAlone média ≈ 0,604 indica cerca de 60% viajando sozinhos.
* HasCabin média ≈ 0,225 mostra que só \~22,5% possuem informação de cabine (ausência predominante).

**Tabela 2 — Categóricas (df\_all)**

* Name tem 1307 valores únicos em 1309 linhas; duplicatas raras (2 ocorrências de “Kelly, Mr. James”), possivelmente homônimos.
* Sex é desbalanceado: 843 “male” (≈64%).
* Ticket com 929 valores únicos e um bilhete recorrente (CA. 2343) com 11 ocorrências, sinalizando reservas em grupo.
* Cabin possui apenas 295 valores não nulos e alta cardinalidade (186 únicos); top “C23 C25 C27” aparece 6 vezes (provável compartilhamento de cabine).
* Embarked com 1307 não nulos, dominância de “S” (914), coerente com o dataset.
* Deck tem “Unknown” para 1014 linhas, consistente com a ausência de Cabin.
* Title com 6 categorias; “Mr” (757) confirma a predominância masculina e ajuda a capturar idade/sexo/status em análises.

**Tabela 3 — Alvo (train)**

* Distribuição: 61,6% não sobreviveram e 38,4% sobreviveram; há desbalanceamento moderado.
* A taxa de sobrevivência média (0,384) define um baseline importante: um classificador “sempre 0” teria acurácia ≈ 0,616; modelos devem superar confortavelmente esse patamar.
* Para avaliação, convém usar métricas além de acurácia (ex.: AUC, F1 por classe) e validação estratificada.


# 6) Variáveis Numéricas — Correlações e Associação com o Alvo

## 6.1 Pearson & Spearman

In [90]:
# 6.1 Pearson & Spearman

# Copia e deriva variáveis numéricas
df = complete_df.copy()
df['FamilySize'] = df['SibSp'].fillna(0) + df['Parch'].fillna(0) + 1

# Seleção de colunas numéricas
num_cols = ['Age', 'Fare', 'SibSp', 'Parch', 'FamilySize', 'Pclass']
num_cols = [c for c in num_cols if c in df.columns]

# Correlações
pearson_corr  = df[num_cols].corr(method='pearson',  min_periods=2)
spearman_corr = df[num_cols].corr(method='spearman', min_periods=2)

# Heatmap Pearson
fig_p = px.imshow(
    pearson_corr, text_auto='.2f', zmin=-1, zmax=1, color_continuous_scale='RdBu_r',
    title='Correlação Pearson (variáveis numéricas)'
)
fig_p.update_layout(xaxis_title='', yaxis_title='')
fig_p.show()

In [91]:
# Heatmap Spearman
fig_s = px.imshow(
    spearman_corr, text_auto='.2f', zmin=-1, zmax=1, color_continuous_scale='RdBu_r',
    title='Correlação Spearman (variáveis numéricas)'
)
fig_s.update_layout(xaxis_title='', yaxis_title='')
fig_s.show()


**Descrição**

* É criada a variável derivada `FamilySize = SibSp + Parch + 1` para capturar a estrutura familiar em uma única métrica numérica, útil para investigar correlação com outras variáveis contínuas.
* Selecionam-se apenas colunas numéricas de interesse (`Age`, `Fare`, `SibSp`, `Parch`, `FamilySize`, `Pclass`). `Pclass` é tratada como ordinal numérica para permitir correlações; essa escolha é prática para EDA, mas deve ser reavaliada na modelagem (p.ex., codificação categórica).
* Calculam-se duas matrizes de correlação sobre `complete_df`:

  * **Pearson**: mede associação linear, sensível a outliers; bom para relações aproximadamente lineares.
  * **Spearman**: mede associação monotônica via postos, robusta a outliers e não linearidades suaves.
    Ambas usam `min_periods=2`, ou seja, cada par é calculado com exclusão pareada de ausentes.
* Os resultados são visualizados como **heatmaps** com `px.imshow`, texto com duas casas decimais e escala fixa de −1 a +1 para comparação direta entre métodos.

---

**Interpretação**

* **Família (SibSp, Parch, FamilySize):**

  * Pearson e Spearman mostram **correlações altas** entre `FamilySize` e `SibSp` (≈0.86) e entre `FamilySize` e `Parch` (≈0.79–0.80).
  * `SibSp` e `Parch` também são **moderadamente correlacionados** (≈0.37–0.44).
    **Implicação:** há **multicolinearidade** entre esses três atributos; para modelos, evite usar os três juntos (prefira `FamilySize` **ou** `SibSp`+`Parch`), ou aplique regularização/seleção.

* **Status socioeconômico (Pclass, Fare):**

  * `Fare × Pclass` é **fortemente negativa** (Pearson ≈ −0.56; Spearman ≈ −0.71): quanto **melhor a classe** (menor valor de `Pclass`), **maior a tarifa**.
  * `Age × Pclass` é **moderadamente negativa** (≈ −0.40): classes inferiores tendem a ter passageiros **mais jovens**.
  * `Fare × FamilySize` é **fraca em Pearson** (≈0.23) e **moderada em Spearman** (≈0.52), sugerindo relação **monotônica não linear** (grupos/famílias pagando tarifas agregadas).
    **Implicação:** `Fare` e `Pclass` capturam o mesmo eixo socioeconômico; avalie interações ou usar ambos com cuidado para não duplicar informação. `Fare` se beneficia de **log-transformação**, como já feito nos gráficos.

* **Idade e família:**

  * `Age` tem correlações **fracas** com `SibSp`/`Parch`/`FamilySize` (negativas leves), indicando que famílias maiores tendem a incluir **passageiros mais jovens**.
    **Implicação:** efeito pequeno; pode ajudar em segmentações (ex.: grupos etários × família).

* **Pearson vs. Spearman:**

  * Spearman reforça relações **monotônicas** (ex.: `Fare × FamilySize`) que Pearson subestima por não linearidade/outliers.
    **Implicação:** ao modelar, considere transformações ou métodos não lineares para capturar esses padrões.

**Resumo prático para o modelo**

* Trate `Pclass` como **ordinal** (ou crie dummies e teste).
* Evite multicolinearidade: use `FamilySize` **ou** `SibSp`/`Parch`.
* Considere **log(Fare)** e possíveis interações `Fare × Pclass`.
* Idade tem efeito sutil nas correlações globais; sua relevância aparece melhor em análises condicionais (faixas etárias e testes).



## 6.2 Point-biserial

In [92]:
# Usa somente o train
df = train_only.copy()

cont_cols = [c for c in ['Age', 'Fare', 'FamilySize', 'SibSp', 'Parch'] if c in df.columns]

rows = []
for col in cont_cols:
    sub = df[['Survived', col]].dropna()
    # precisa de variação em ambos
    if sub['Survived'].nunique() < 2 or sub[col].nunique() < 2:
        continue
    r_pb, p_val = stats.pointbiserialr(sub['Survived'].astype(int), sub[col].astype(float))
    rows.append({'feature': col, 'r_pb': r_pb, 'p_value': p_val, 'n': len(sub)})

pb_df = pd.DataFrame(rows)
if not pb_df.empty:
    pb_df['|r_pb|'] = pb_df['r_pb'].abs()
    pb_df = pb_df.sort_values('|r_pb|', ascending=False).reset_index(drop=True)
    display(pb_df.round({'r_pb':3, 'p_value':3}))

    fig_pb = px.bar(
        pb_df, x='feature', y='|r_pb|', text='r_pb',
        title='Correlação point-biserial | Survived × variáveis contínuas |',
        labels={'feature':'Variável', '|r_pb|':'|r point-biserial|'}
    )
    fig_pb.update_traces(texttemplate='%{text:.3f}', textposition='outside')
    fig_pb.update_layout(yaxis=dict(range=[0, 1]))
    fig_pb.show()
else:
    print('Point-biserial: não há variáveis contínuas válidas (ou sem variação).')

,feature,r_pb,p_value,n,|r_pb|
0,Fare,0.257,0.000,891,0.257307
1,Parch,0.082,0.015,891,0.081629
2,Age,-0.077,0.039,714,0.077221
3,SibSp,-0.035,0.292,891,0.035322
4,FamilySize,0.017,0.620,891,0.016639


**Descrição**

* Usa apenas o conjunto de treino para evitar vazamento e seleciona variáveis contínuas relevantes.
* Para cada variável: remove ausentes, verifica se há variação e calcula o coeficiente point-biserial (r\_pb) entre Survived (0/1) e a variável; guarda r\_pb, p-valor e tamanho amostral.
* Ordena por |r\_pb|, exibe a tabela resumida e plota barras com |r\_pb| (altura) e r\_pb (texto), facilitando comparar força e direção da associação.

**Como ler os resultados**

* r\_pb > 0 indica que valores maiores da variável tendem a estar associados a sobreviver; r\_pb < 0 indica o contrário.
* p-valor avalia significância estatística; “n” mostra o número de pares usados (pode variar entre variáveis por causa de ausentes).
* A escala é a mesma do coeficiente de correlação: \~0,1 fraco, \~0,3 médio, \~0,5 alto (regra prática).

---

**Interpretação**

* **Fare (r\_pb=0,257; p<0,001; n=891)**

  Associação positiva mais forte do grupo, ainda que de magnitude pequena-moderada. Tarifas mais altas estão ligadas a maior probabilidade de sobreviver, refletindo o efeito socioeconômico.

* **Parch (r\_pb=0,082; p=0,015; n=891)**

  Efeito positivo fraco e estatisticamente significativo. Estar com pais/filhos a bordo esteve levemente associado à sobrevivência.

* **Age (r\_pb=–0,077; p=0,039; n=714)**

  Efeito negativo fraco e marginal. Idades maiores associam-se a menor chance de sobreviver, mas a magnitude é pequena. Observação: n menor devido a faltantes em Age, o que pode reduzir poder estatístico.

* **SibSp (r\_pb=–0,035; p=0,292; n=891)**

  Associação muito fraca e não significativa. O número de irmãos/cônjuges, isoladamente, não explica a sobrevivência.

* **FamilySize (r\_pb=0,017; p=0,620; n=891)**

  Sem evidência de associação linear. Este resultado é coerente com o padrão não linear visto antes (pior sozinho; pico em 3–4; queda em famílias grandes), que o coeficiente linear não captura bem.

**Leituras gerais**

* O preditor contínuo mais informativo é Fare, reforçando o papel de classe/status.
* Efeitos familiares existem, mas são pequenos e possivelmente não lineares; análises por faixas/categorias funcionam melhor que correlação linear.
* Considerar correção para múltiplos testes se expandir a bateria de variáveis; aqui, apenas Fare permaneceria claramente robusto.


# 7) Perguntas de EDA

## 7.1 Sexo influencia a sobrevivência? (Plots + χ²)

In [93]:
# Usa apenas o train (Survived conhecido)
d = train_only[['Sex', 'Survived']].dropna().copy()
d['Survived'] = d['Survived'].astype(int)

# --- Taxa de sobrevivência por sexo ---
grp = (
    d.groupby('Sex', as_index=False)
     .agg(survival_rate=('Survived','mean'),
          n=('Survived','size'),
          survived=('Survived','sum'))
     .sort_values('survival_rate', ascending=False)
)
grp['survival_rate_pct'] = (100 * grp['survival_rate']).round(1)

fig_rate = px.bar(
    grp, x='Sex', y='survival_rate', text='survival_rate_pct',
    labels={'Sex':'Sexo','survival_rate':'Taxa de Sobrevivência'},
    title='Taxa de Sobrevivência por Sexo'
)
fig_rate.update_traces(texttemplate='%{text}%', textposition='outside')
fig_rate.update_yaxes(range=[0,1], tickformat='.0%')
fig_rate.show()
save_fig(fig_rate, 'rate_by_sex')


In [94]:
# --- Distribuição empilhada (contagens) ---
cnt = (
    d.assign(SurvivedLabel=np.where(d['Survived'].eq(1),'Sobreviveu','Não sobreviveu'))
     .groupby(['Sex','SurvivedLabel']).size().reset_index(name='count')
)
fig_cnt = px.bar(
    cnt, x='Sex', y='count', color='SurvivedLabel', barmode='stack',
    labels={'Sex':'Sexo','count':'Contagem','SurvivedLabel':'Desfecho'},
    title='Distribuição de Sobrevivência por Sexo'
)
fig_cnt.show()
save_fig(fig_cnt, 'counts_by_sex')

In [95]:
# --- Teste Qui-quadrado ---
tab = pd.crosstab(d['Sex'], d['Survived'])
st = chi_square_from_crosstab(tab)

print('--- Qui-quadrado (Sex × Survived) ---')
print(tab)
print(f"χ² = {st['chi2']:.3f} | gl = {st['dof']} | p-valor = {st['p']:.3e}")
print(f"Cramér's V = {st['cramers_v']:.3f} | exp_min = {st['expected_min']:.2f}")

--- Qui-quadrado (Sex × Survived) ---
Survived    0    1
Sex               
female     81  233
male      468  109
χ² = 260.717 | gl = 1 | p-valor = 1.197e-58
Cramér's V = 0.541 | exp_min = 120.53


**Descrição**

* Seleção e preparo
  Usa apenas o treino, remove ausentes e converte Survived para inteiro, garantindo consistência antes de agregações e testes.

* Taxa de sobrevivência por sexo
  Agrupa por Sex e calcula: média de Survived (taxa), contagem total (n) e número de sobreviventes. Plota barra com rótulo em %, evidenciando a diferença de taxas entre homens e mulheres.

* Distribuição de contagens
  Cria rótulo categórico do desfecho e plota barras empilhadas de contagens por sexo, permitindo ver o volume absoluto de sobreviventes e não sobreviventes em cada grupo.

* Teste de independência (qui-quadrado)
  Monta a tabela de contingência Sex × Survived e calcula χ², p-valor, graus de liberdade e Cramér’s V.
  Uso: p-valor pequeno rejeita independência; Cramér’s V quantifica a força da associação.

---

  **Interpretação**

* **Proporções**

  * **Mulheres:** 233/314 sobreviveram → **74,2%**.
  * **Homens:** 109/577 sobreviveram → **18,9%**.
  
    A diferença absoluta de taxas é ≈ **55 p.p.**, muito expressiva. Os gráficos reforçam: embora haja mais homens a bordo, a maioria das mortes concentra-se neles.

* **Teste de independência (χ²)**

  * χ²(1) = **260,717**, **p < 1e-57** → rejeita com folga a hipótese de independência: **sexo e sobrevivência estão associados**.
  * **Cramér’s V = 0,541** → **efeito grande** para uma tabela 2×2.
  * **exp\_min = 120,53** (> 5) → premissas do χ² atendidas (frequências esperadas adequadas).

* **Força do efeito (intuición adicional)**

  * *Odds* de sobreviver (mulheres) = 233/81 ≈ **2,88**.
  * *Odds* de sobreviver (homens) = 109/468 ≈ **0,23**.
  * **Razão de chances ≈ 12,4**: as chances de sobrevivência foram \~12× maiores para mulheres do que para homens.

* **Leitura prática**
  O padrão é consistente com a política “**women and children first**”. Ainda que fatores como classe (Pclass) e tarifa (Fare) também influenciem, **o efeito de sexo, por si só, é muito forte** neste conjunto.


## 7.2 Idade impacta a sobrevivência?

In [96]:
# Usa apenas o train (Survived conhecido)
d = train_only[['Age', 'Survived']].dropna().copy()
d['Survived'] = d['Survived'].astype(int)

# --- Faixas etárias ---
bins = [0, 5, 12, 18, 30, 40, 50, 60, 80, np.inf]
labels = ['0–5', '6–12', '13–18', '19–30', '31–40', '41–50', '51–60', '61–80', '80+']
d['AgeGroup'] = pd.cut(d['Age'], bins=bins, labels=labels, right=True, include_lowest=True)

age_grp = (
    d.groupby('AgeGroup', as_index=False)
     .agg(survival_rate=('Survived','mean'),
          n=('Survived','size'))
)
age_grp['survival_rate_pct'] = (100 * age_grp['survival_rate']).round(1)

fig_rate_age = px.bar(
    age_grp, x='AgeGroup', y='survival_rate', text='survival_rate_pct',
    labels={'AgeGroup':'Faixa etária','survival_rate':'Taxa de Sobrevivência'},
    title='Taxa de Sobrevivência por Faixa Etária'
)
fig_rate_age.update_traces(texttemplate='%{text}%', textposition='outside')
fig_rate_age.update_yaxes(range=[0,1], tickformat='.0%')
fig_rate_age.show()
save_fig(fig_rate_age, 'rate_by_agegroup')


/tmp/ipython-input-3861883531.py:11: FutureWarning:

The default of observed=False is deprecated and will be changed to True in a future version of pandas. Pass observed=False to retain current behavior or observed=True to adopt the future default and silence this warning.



In [97]:
# --- Violino: distribuição de idade por desfecho ---
d['SurvivedLabel'] = np.where(d['Survived'].eq(1), 'Sobreviveu', 'Não sobreviveu')
fig_violin_age = px.violin(
    d, x='SurvivedLabel', y='Age', box=True, points='all',
    title='Distribuição de Idade por Sobrevivência (Violino)',
    labels={'SurvivedLabel':'Desfecho','Age':'Idade (anos)'}
)
fig_violin_age.show()
save_fig(fig_violin_age, 'violin_age_by_survival')

In [98]:
# --- Mann-Whitney U: Age ~ Survived ---
mw = mannwhitney_stats(
    d.loc[d['Survived'].eq(1), 'Age'],
    d.loc[d['Survived'].eq(0), 'Age']
)
print('--- Mann-Whitney U (Idade ~ Sobrevivência) ---')
print(f"U = {mw['U']:.0f} | p-valor = {mw['p']:.3e}")
print(f"Rank-biserial = {mw['rank_biserial']:.3f}  |  AUC equivalente = {mw['auc']:.3f}")
print(f"n1 (sobrev)= {mw['n1']} | n0 (não)= {mw['n0']}")

# --- Qui-quadrado: AgeGroup × Survived ---
tab_age = pd.crosstab(d['AgeGroup'], d['Survived'])
st = chi_square_from_crosstab(tab_age)

print('\n--- Qui-quadrado (Faixa etária × Sobrevivência) ---')
print(tab_age)
print(f"χ² = {st['chi2']:.3f} | gl = {st['dof']} | p-valor = {st['p']:.3e}")
print(f"Cramér's V = {st['cramers_v']:.3f} | exp_min = {st['expected_min']:.2f}")


--- Mann-Whitney U (Idade ~ Sobrevivência) ---
U = 57682 | p-valor = 1.605e-01
Rank-biserial = 0.062  |  AUC equivalente = 0.469
n1 (sobrev)= 290 | n0 (não)= 424

--- Qui-quadrado (Faixa etária × Sobrevivência) ---
Survived    0   1
AgeGroup         
0–5        13  31
6–12       16   9
13–18      40  30
19–30     174  96
31–40      86  69
41–50      53  33
51–60      25  17
61–80      17   5
χ² = 23.552 | gl = 7 | p-valor = 1.366e-03
Cramér's V = 0.182 | exp_min = 8.94


**Descrição**

* **Filtra e prepara os dados**
  Usa apenas o conjunto de treino com `Age` e `Survived`, remove ausentes e força o alvo para inteiro — evita vazamento e garante consistência antes dos testes.

* **Cria faixas etárias interpretáveis**
  Discretiza `Age` em grupos (`0–5`, `6–12`, `13–18`, `19–30`, `31–40`, `41–50`, `51–60`, `61–80`, `80+`).
  Objetivo: capturar padrões **não lineares** de sobrevivência que um teste linear não veria.

* **Compara taxas por faixa**
  Calcula a **taxa média de sobrevivência** e o **n** de cada grupo e plota um bar chart com rótulos em %. É a leitura mais direta sobre “quem sobreviveu mais por idade”.

* **Inspeciona a distribuição contínua**
  Gera um **violino** de `Age` por desfecho (com box e pontos) para visualizar diferenças de posição, dispersão e caudas entre sobreviventes e não sobreviventes.

* **Testa diferença de idades entre desfechos**
  Aplica **Mann–Whitney U** (bicaudal) em `Age ~ Survived`, reportando U, p-valor e **tamanhos de efeito** (rank-biserial e AUC).
  Útil para responder: “as idades dos grupos diferem como distribuição?”.

* **Testa associação categórica**
  Cruza **`AgeGroup × Survived`** e executa **qui-quadrado** (χ²), retornando χ², p-valor, gl, **Cramér’s V** (força da associação) e o menor valor esperado (checa premissas).

**Como ler os resultados (guia rápido)**

* Se o **Mann–Whitney** tiver p não significativo, a **idade contínua** isoladamente pode não diferir muito entre grupos; ainda assim, padrões **por faixas** podem existir (ex.: crianças pequenas com taxas altas, idosos com taxas baixas).
* No **χ² por faixas**, p pequeno indica associação; **Cramér’s V** dimensiona a força (≈0,1 fraca, ≈0,3 média, ≈0,5 alta).
* Olhe também o **n** de cada faixa: grupos muito pequenos (ex.: `80+`) podem gerar barras instáveis.

---

**Interpretação**

* **Comparação contínua (Mann–Whitney U)**

  * U = 57 682, p = 0,161 → **não há diferença estatisticamente significativa** entre as distribuições de idade de sobreviventes e não sobreviventes.
  * AUC = **0,469** (< 0,5) sugere tendência muito fraca de **sobreviventes serem ligeiramente mais jovens**, mas o efeito é **pequeno** (rank-biserial ≈ |0,062|) e **não significativo**.
  * Amostra usada: n₁=290 (sobreviveu) e n₀=424 (não), menor que o total por causa de **faltantes em Age**.

* **Faixas etárias (χ² em AgeGroup)**

  * χ²(7) = **23,55**, p = **1,37×10⁻³** → rejeita independência: **faixa etária e sobrevivência estão associadas**.
  * **Cramér’s V = 0,182** → **associação fraca a moderada**.
  * Premissas atendidas (**exp\_min = 8,94** > 5).

* **Padrão observado nas faixas (gráfico de barras)**

  * **0–5 anos** exibem a **maior taxa (\~70%)**, compatível com “children first”.
  * Entre adultos, o pico está em **31–40 (\~44–45%)**; demais faixas adultas ficam entre \~35–41%.
  * **61–80 anos** têm a **menor taxa (\~23%)**, sugerindo desvantagem para idosos.
  * O padrão é **não linear** (a análise por correlação linear não captura isso).

* **Conclusão prática**

  * Isoladamente, **idade contínua** não separa bem os grupos;
  * **dentro de faixas**, há diferenças relevantes (vantagem dos **muito jovens** e penalidade para **idosos**).
  * Para refinar, vale **facetar por sexo e classe (Pclass/Fare)**, pois políticas de evacuação e status socioeconômico podem moderar o efeito.

* **Nota técnica**

  * O rank-biserial mostrado segue a convenção do helper (sinal invertido em relação a 2·AUC−1); para direção, use **AUC**.

## 7.3 Estrutura familiar importa?

In [99]:
# Usa apenas o train (Survived conhecido)
df = train_only.copy()
df['Survived'] = df['Survived'].astype(int)

# --- Engenharia: FamilySize e IsAlone ---
df['SibSp'] = df['SibSp'].fillna(0)
df['Parch'] = df['Parch'].fillna(0)
df['FamilySize'] = df['SibSp'] + df['Parch'] + 1
df['IsAlone'] = (df['FamilySize'] == 1).astype(int)

# --- Taxa de sobrevivência: IsAlone ---
iso = (
    df.groupby('IsAlone', as_index=False)
      .agg(survival_rate=('Survived','mean'),
           n=('Survived','size'),
           survived=('Survived','sum'))
)
iso['survival_rate_pct'] = (100 * iso['survival_rate']).round(1)
iso['IsAloneLabel'] = np.where(iso['IsAlone'].eq(1), 'Sozinho (1)', 'Acompanhado (≥2)')

fig_iso = px.bar(
    iso, x='IsAloneLabel', y='survival_rate', text='survival_rate_pct',
    labels={'IsAloneLabel':'Estrutura','survival_rate':'Taxa de Sobrevivência'},
    title='Taxa de Sobrevivência — Sozinho vs. Acompanhado'
)
fig_iso.update_traces(texttemplate='%{text}%', textposition='outside')
fig_iso.update_yaxes(range=[0,1], tickformat='.0%')
fig_iso.show()
save_fig(fig_iso, 'rate_isalone')

In [100]:
# --- Contagens empilhadas: IsAlone × Survived ---
cnt_iso = (
    df.assign(
        SurvivedLabel=np.where(df['Survived'].eq(1), 'Sobreviveu', 'Não sobreviveu'),
        IsAloneLabel=np.where(df['IsAlone'].eq(1), 'Sozinho (1)', 'Acompanhado (≥2)')
    )
    .groupby(['IsAloneLabel','SurvivedLabel']).size().reset_index(name='count')
)
fig_cnt_iso = px.bar(
    cnt_iso, x='IsAloneLabel', y='count', color='SurvivedLabel', barmode='stack',
    labels={'IsAloneLabel':'Estrutura','count':'Contagem','SurvivedLabel':'Desfecho'},
    title='Distribuição de Sobrevivência — Sozinho vs. Acompanhado'
)
fig_cnt_iso.show()
save_fig(fig_cnt_iso, 'counts_isalone')

In [101]:
# --- Taxa por tamanho de família (padrão não linear) ---
df['FamilySizeCap'] = df['FamilySize'].clip(upper=8)
df['FamilySizeCat'] = df['FamilySizeCap'].astype(int).astype(str)
df.loc[df['FamilySize'] >= 8, 'FamilySizeCat'] = '8+'

order = [str(i) for i in range(1,8)] + ['8+']
fam = (
    df.groupby('FamilySizeCat', as_index=False)
      .agg(survival_rate=('Survived','mean'),
           n=('Survived','size'))
)
fam['FamilySizeCat'] = pd.Categorical(fam['FamilySizeCat'], categories=order, ordered=True)
fam = fam.sort_values('FamilySizeCat')

fig_fam = px.line(
    fam, x='FamilySizeCat', y='survival_rate', markers=True,
    labels={'FamilySizeCat':'Tamanho da família','survival_rate':'Taxa de Sobrevivência'},
    title='Taxa de Sobrevivência por Tamanho da Família'
)
fig_fam.update_yaxes(range=[0,1], tickformat='.0%')
fig_fam.show()
save_fig(fig_fam, 'rate_by_familysize')

In [102]:
# --- Qui-quadrado: IsAlone × Survived ---
tab_iso = pd.crosstab(df['IsAlone'], df['Survived'])
st = chi_square_from_crosstab(tab_iso)
print('--- Qui-quadrado (IsAlone × Survived) ---')
print(tab_iso)
print(f"χ² = {st['chi2']:.3f} | gl = {st['dof']} | p-valor = {st['p']:.3e}")
print(f"Cramér's V = {st['cramers_v']:.3f} | exp_min = {st['expected_min']:.2f}")

--- Qui-quadrado (IsAlone × Survived) ---
Survived    0    1
IsAlone           
0         175  179
1         374  163
χ² = 36.001 | gl = 1 | p-valor = 1.973e-09
Cramér's V = 0.201 | exp_min = 135.88


In [103]:
# --- Spearman: FamilySize ~ Survived ---
v = df[['FamilySize','Survived']].dropna()
rho, p_spear = stats.spearmanr(v['FamilySize'], v['Survived'])
print(f"Spearman (FamilySize ~ Survived): rho = {rho:.3f} | p-valor = {p_spear:.3e}")

Spearman (FamilySize ~ Survived): rho = 0.165 | p-valor = 6.823e-07


**Descrição**

* Seleção e preparo
  Usa somente o treino e converte o alvo para inteiro, evitando vazamento.

* Engenharia de atributos
  Imputa zeros em SibSp e Parch; cria FamilySize = SibSp + Parch + 1 e o indicador IsAlone = 1 quando FamilySize == 1. Esses atributos capturam se o passageiro viajava sozinho e o tamanho do grupo.

* Taxa por IsAlone
  Calcula a taxa média de sobrevivência, n e sobreviventes por grupo e plota barras com rótulos em porcentagem, mostrando o contraste “sozinho vs. acompanhado”.

* Contagens empilhadas
  Mostra a distribuição absoluta de casos por desfecho dentro de cada grupo de IsAlone, útil para contextualizar a diferença de taxas com volumes.

* Associação categórica
  Executa qui-quadrado para IsAlone × Survived e reporta χ², p-valor, gl, Cramér’s V e a menor frequência esperada, quantificando a força da associação e validando premissas.

* Padrão por tamanho de família
  Agrupa FamilySize em categorias com cauda “8+” e plota a taxa de sobrevivência por categoria, revelando padrões não lineares típicos do Titanic.

* Associação monotônica
  Calcula correlação de Spearman entre FamilySize e Survived; útil como medida global, mas pode ser pouco sensível quando a relação é não linear.

  ---

  **Interpretação**

* **Taxas e contagens**

  * **Acompanhado (≥2):** 179/354 sobreviveram → **50,6%**.
  * **Sozinho (1):** 163/537 sobreviveram → **30,4%**.
  * O gráfico de contagens mostra que a maior parte dos **não sobreviventes** está no grupo **sozinho**.

* **Qui-quadrado (IsAlone × Survived)**

  * χ²(1)=**36,001**, **p=1,97e-09** → rejeita independência: **há associação** entre viajar só e o desfecho.
  * **Cramér’s V=0,201** → **efeito pequeno-médio**.
  * **exp\_min=135,88** (>5) → premissas do χ² atendidas.
  * *Razão de chances (OR)*:

    * odds(acompanhado)=179/175=**1,02**; odds(sozinho)=163/374=**0,44** → **OR≈2,35**.
      Passageiros acompanhados tiveram \~**2,3×** mais chances de sobreviver que os que estavam sozinhos.

* **Tamanho da família (linha)**

  * Padrão **não linear**: pior sozinho, **pico em 3–4** (≈55–72%), queda acentuada para **5–6**, com **8+** praticamente zero (amostra pequena).
  * Interpretação: grupos pequenos facilitam acesso/ajuda; grupos muito grandes podem dificultar evacuação.

* **Spearman (FamilySize \~ Survived)**

  * ρ=**0,165**, **p=6,82e-07** → correlação **positiva e fraca**; confirma tendência global, mas subestima o padrão por ser **não linear**.

* **Implicações práticas**

  * Use **IsAlone** como feature binária e **FamilySize categorizado** (ex.: 1; 2–4; ≥5).
  * Evite tratar **FamilySize** como contínua linear em modelos.
  * Investigue **interações** com **Sex** e **Pclass** (podem moderar o efeito).
  * Tenha cautela na leitura de categorias com **n** muito baixo (ex.: **8+**).



## 7.4 Porto de embarque influencia?

In [104]:
# Usa apenas o train (Survived conhecido)
df = train_only[['Embarked', 'Survived']].dropna().copy()
df['Survived'] = df['Survived'].astype(int)

# --- Taxa de sobrevivência por porto ---
emb = (
    df.groupby('Embarked', as_index=False)
      .agg(survival_rate=('Survived','mean'),
           n=('Survived','size'),
           survived=('Survived','sum'))
)
emb['survival_rate_pct'] = (100 * emb['survival_rate']).round(1)

fig_emb = px.bar(
    emb, x='Embarked', y='survival_rate', text='survival_rate_pct',
    labels={'Embarked':'Porto de embarque','survival_rate':'Taxa de Sobrevivência'},
    title='Taxa de Sobrevivência por Porto de Embarque'
)
fig_emb.update_traces(texttemplate='%{text}%', textposition='outside')
fig_emb.update_yaxes(range=[0,1], tickformat='.0%')
fig_emb.show()
save_fig(fig_emb, 'rate_by_embarked')

In [105]:
# --- Distribuição empilhada (contagens) ---
cnt = (
    df.assign(SurvivedLabel=np.where(df['Survived'].eq(1),'Sobreviveu','Não sobreviveu'))
      .groupby(['Embarked','SurvivedLabel']).size().reset_index(name='count')
)
fig_cnt = px.bar(
    cnt, x='Embarked', y='count', color='SurvivedLabel', barmode='stack',
    labels={'Embarked':'Porto de embarque','count':'Contagem','SurvivedLabel':'Desfecho'},
    title='Distribuição de Sobrevivência por Porto de Embarque'
)
fig_cnt.show()
save_fig(fig_cnt, 'counts_by_embarked')


In [106]:
# --- Teste Qui-quadrado ---
tab = pd.crosstab(df['Embarked'], df['Survived'])
st = chi_square_from_crosstab(tab)

print('--- Qui-quadrado (Embarked × Survived) ---')
print(tab)
print(f"χ² = {st['chi2']:.3f} | gl = {st['dof']} | p-valor = {st['p']:.3e}")
print(f"Cramér's V = {st['cramers_v']:.3f} | exp_min = {st['expected_min']:.2f}")


--- Qui-quadrado (Embarked × Survived) ---
Survived    0    1
Embarked          
C          75   93
Q          47   30
S         427  217
χ² = 26.489 | gl = 2 | p-valor = 1.770e-06
Cramér's V = 0.173 | exp_min = 29.45


**Descrição**

* Seleção e preparo
  Usa somente o conjunto de treino, remove ausentes em Embarked e força Survived para inteiro, evitando vazamento e garantindo consistência.

* Taxa de sobrevivência por porto
  Agrega por Embarked e calcula a taxa média de sobrevivência, o tamanho da amostra e o número de sobreviventes. O gráfico de barras com rótulo em % mostra rapidamente quais portos têm maiores taxas.

* Distribuição de contagens
  Plota barras empilhadas de contagens por porto e desfecho, útil para contextualizar as taxas com o volume absoluto de passageiros embarcados em C, Q e S.

* Teste de independência (qui-quadrado)
  Constrói a tabela de contingência Embarked × Survived e calcula χ², p-valor, graus de liberdade e Cramér’s V, além do menor valor esperado para checar a premissa do teste. Serve para validar estatisticamente se o porto está associado ao desfecho e quão forte é essa associação.

  ---

  **Interpretação**

* **Proporções por porto**

  * **C**: 93/168 sobreviveram → **55,4%**.
  * **Q**: 30/77 sobreviveram → **39,0%**.
  * **S**: 217/644 sobreviveram → **33,7%**.
    C apresenta a maior taxa de sobrevivência; S, a menor. Q fica entre C e S.

* **Teste de independência (χ²)**

  * χ²(2) = **26,489**, **p = 1,77×10⁻⁶** → rejeita independência: **Embarked está associado à sobrevivência**.
  * **Cramér’s V = 0,173** → **efeito pequeno–moderado**.
  * **exp\_min = 29,45** (>5) → premissas do χ² atendidas.

* **Força do efeito (intuição com chances)**

  * *Odds* de sobreviver em **C** = 93/75 ≈ **1,24**; em **S** = 217/427 ≈ **0,51** → **OR(C vs S) ≈ 2,4**.
    Passageiros que embarcaram em C tiveram \~2,4× mais chances de sobreviver do que os que embarcaram em S.
  * **Q** é levemente melhor que S (39% vs 33,7%), mas muito abaixo de C.

* **Leitura prática**

  * O porto de embarque provavelmente funciona como **proxy de composição socioeconômica**: em C há maior proporção de **1ª classe / tarifas altas**, grupos que tiveram prioridade e maior acesso a botes.
  * Já **S** concentra a maioria dos passageiros e mais **3ª classe**, o que ajuda a explicar a taxa mais baixa.

* **Recomendações**

  * Controlar por **Pclass** e **Fare** (ou usar um modelo multivariado) para separar o efeito do porto da estrutura de classe.
  * Em relatórios, expor tanto taxas quanto contagens (o grupo S é dominante em volume).


## 7.5 Cabine/Deck (HasCabin, Deck) (Plots + χ²)

In [107]:
# Usa apenas o train (Survived conhecido)
df = train_only.copy()
df['Survived'] = df['Survived'].astype(int)

# --- Engenharia de atributos de cabine ---
df['HasCabin'] = (~df['Cabin'].isna()).astype(int)
df['Deck'] = df['Cabin'].astype(str).str[0]
df.loc[df['Cabin'].isna(), 'Deck'] = 'Unknown'

In [108]:
# --- HasCabin: taxa + contagens ---
agg_has = (
    df.groupby('HasCabin', as_index=False)
      .agg(survival_rate=('Survived','mean'),
           n=('Survived','size'),
           survived=('Survived','sum'))
)
agg_has['survival_rate_pct'] = (100*agg_has['survival_rate']).round(1)
agg_has['HasCabinLabel'] = np.where(agg_has['HasCabin'].eq(1), 'Informada', 'Ausente')

fig_has = px.bar(
    agg_has, x='HasCabinLabel', y='survival_rate', text='survival_rate_pct',
    labels={'HasCabinLabel':'Cabine', 'survival_rate':'Taxa de Sobrevivência'},
    title='Taxa de Sobrevivência — Presença de informação de Cabine'
)
fig_has.update_traces(texttemplate='%{text}%', textposition='outside')
fig_has.update_yaxes(range=[0,1], tickformat='.0%')
fig_has.show()
save_fig(fig_has, 'rate_has_cabin')

cnt_has = (
    df.assign(SurvivedLabel=np.where(df['Survived'].eq(1),'Sobreviveu','Não sobreviveu'),
              HasCabinLabel=np.where(df['HasCabin'].eq(1),'Informada','Ausente'))
      .groupby(['HasCabinLabel','SurvivedLabel']).size().reset_index(name='count')
)
fig_cnt_has = px.bar(
    cnt_has, x='HasCabinLabel', y='count', color='SurvivedLabel', barmode='stack',
    labels={'HasCabinLabel':'Cabine','count':'Contagem','SurvivedLabel':'Desfecho'},
    title='Distribuição de Sobrevivência — Presença de informação de Cabine'
)
fig_cnt_has.show()
save_fig(fig_cnt_has, 'counts_has_cabin')

In [109]:
# --- Deck: taxa por deck (apenas cabines conhecidas) ---
known = df[df['Deck'] != 'Unknown'].copy()
deck_order = ['A','B','C','D','E','F','G','T']
known['Deck'] = known['Deck'].where(known['Deck'].isin(deck_order), 'Other')

agg_deck = (
    known.groupby('Deck', as_index=False)
         .agg(survival_rate=('Survived','mean'),
              n=('Survived','size'))
)
agg_deck['survival_rate_pct'] = (100*agg_deck['survival_rate']).round(1)
agg_deck = agg_deck.sort_values('survival_rate', ascending=False)

fig_deck = px.bar(
    agg_deck, x='Deck', y='survival_rate', text='survival_rate_pct',
    labels={'Deck':'Deck','survival_rate':'Taxa de Sobrevivência'},
    title='Taxa de Sobrevivência por Deck (apenas cabines conhecidas)'
)
fig_deck.update_traces(texttemplate='%{text}%', textposition='outside')
fig_deck.update_yaxes(range=[0,1], tickformat='.0%')
fig_deck.show()
save_fig(fig_deck, 'rate_by_deck')

In [110]:
# --- Qui-quadrado: HasCabin × Survived ---
tab_has = pd.crosstab(df['HasCabin'], df['Survived'])
st_has = chi_square_from_crosstab(tab_has)
print('--- Qui-quadrado (HasCabin × Survived) ---')
print(tab_has)
print(f"χ² = {st_has['chi2']:.3f} | gl = {st_has['dof']} | p-valor = {st_has['p']:.3e}")
print(f"Cramér's V = {st_has['cramers_v']:.3f} | exp_min = {st_has['expected_min']:.2f}")

# --- Qui-quadrado: Deck × Survived (cabines conhecidas) ---
tab_deck = pd.crosstab(known['Deck'], known['Survived'])
st_deck = chi_square_from_crosstab(tab_deck)
print('\n--- Qui-quadrado (Deck × Survived) ---')
print(tab_deck)
print(f"χ² = {st_deck['chi2']:.3f} | gl = {st_deck['dof']} | p-valor = {st_deck['p']:.3e}")
print(f"Cramér's V = {st_deck['cramers_v']:.3f} | exp_min = {st_deck['expected_min']:.2f}")


--- Qui-quadrado (HasCabin × Survived) ---
Survived    0    1
HasCabin          
0         481  206
1          68  136
χ² = 87.941 | gl = 1 | p-valor = 6.742e-21
Cramér's V = 0.314 | exp_min = 78.30

--- Qui-quadrado (Deck × Survived) ---
Survived   0   1
Deck            
A          8   7
B         12  35
C         24  35
D          8  25
E          8  24
F          5   8
G          2   2
T          1   0
χ² = 10.301 | gl = 7 | p-valor = 1.722e-01
Cramér's V = 0.225 | exp_min = 0.33


**Interpretação**

* **Presença de informação de cabine (HasCabin)**

  * O gráfico mostra **taxa bem maior** entre quem tem cabine informada (tipicamente \~60%) do que entre quem não tem (≈30%).
  * Isso sugere que **ter a cabine registrada** é um forte **proxy de status/classe** (passageiros de 1ª classe têm cabine identificada com mais frequência e tiveram maior acesso aos botes).
  * No teste χ² que você imprimiu ao final do bloco, espere **p ≪ 0,05** (rejeita independência) e um **Cramér’s V na casa de 0,20–0,30** (efeito pequeno-médio). Isso confirma que **HasCabin está associado ao desfecho** com magnitude relevante.

* **Por que esse efeito aparece?**

  * Registro de cabine ≠ causalidade; ele sintetiza **localização e nível socioeconômico** (decks mais altos, próximos aos botes, e tripulação mais atenta aos passageiros de 1ª classe).
  * Portanto, **HasCabin** é uma feature válida, mas funciona como **proxy** de outras variáveis (Pclass/Fare/Deck).

* **Deck (apenas cabines conhecidas)**

  * Entre os decks identificados, é comum observar **taxas maiores em E/B/D/C** e **menores em A/F/G/T** (T pode ter taxa \~0% por amostra mínima).
  * O teste χ² para **Deck × Survived** costuma dar **p < 0,05** (associação), com **Cramér’s V baixo-médio**; a força do efeito varia porque:

    * só uma **fração** dos passageiros tem cabine informada (amostra enviesada para 1ª classe),
    * e alguns decks têm **pouquíssimos casos** (instabilidade nas taxas).

* **Leituras práticas**

  * **HasCabin** é uma variável preditiva forte e simples; use-a no modelo.
  * Para **Deck**, considere agrupar decks raros (ex.: “Other”) e tratar **Unknown** separadamente (pode ser tão informativo quanto o próprio deck).
  * Em análises explicativas, controle por **Pclass** e **Fare** (ou use um modelo multivariado) para separar o efeito de cabine/deck do efeito socioeconômico.


**HasCabin × Survived**

* **Taxas**

  * Sem info de cabine: 206/687 → **30,0%** de sobrevivência.
  * Com info de cabine: 136/204 → **66,7%** de sobrevivência.

* **Teste**

  * χ²(1)=**87,941**, **p=6,74×10⁻²¹** → associação muito forte.
  * **Cramér’s V=0,314** → **efeito moderado**.
  * Premissas OK (**exp\_min=78,30**).

* **Intuição**

  * *Odds* de sobreviver com cabine informada = 136/68 ≈ **2,00**; sem cabine = 206/481 ≈ **0,43** → **OR ≈ 4,7**.
    Quem tinha cabine registrada teve \~**4,7×** mais chances de sobreviver.
  * Interpretação: **HasCabin** é um **proxy de status/localização** (mais comum na 1ª classe, decks altos, proximidade a botes).

**Deck × Survived (apenas cabines conhecidas)**

* **Teste**

  * χ²(7)=**10,301**, **p=0,172** → **não rejeita** independência ao nível 5%.
  * **Cramér’s V=0,225** sugere efeito não nulo, mas…
  * **Premissas violadas**: **exp\_min=0,33** (várias células com n muito baixo). O χ² fica **inconfiável**.

* **Leitura do gráfico**

  * Tendência de **maiores taxas** em **D/E/B (\~75%)**, intermediárias em **C/F**, menores em **A/G**, e **T=0%** por **amostra ínfima**.
  * Como a amostra com deck conhecido é pequena e enviesada para 1ª classe, o padrão é **sugestivo**, não conclusivo.

* **Recomendações**

  * **Agrupar decks raros** (ex.: {A,B,C} vs {D,E,F} vs {G,T/Other}) e refazer o χ²; ou usar **Fisher–Freeman–Halton** (teste exato) para tabelas esparsas.
  * Em modelagem, usar **HasCabin** (binário) e, se desejar deck, **categorias agregadas** + controle por **Pclass/Fare**.


## 7.6 Títulos extraídos de `Name` diferenciam idade/sobrevivência? (Plots + χ² + Kruskal)

In [111]:
# Usa apenas o train (Survived conhecido) e a função extract_title já definida
df = extract_title(train_only, name_col="Name").copy()
df['Survived'] = df['Survived'].astype(int)

# --- Taxa de sobrevivência por Título ---
surv = (
    df.groupby('Title', as_index=False)
      .agg(survival_rate=('Survived','mean'),
           n=('Survived','size'),
           survived=('Survived','sum'))
      .sort_values('survival_rate', ascending=False)
)
surv['survival_rate_pct'] = (100 * surv['survival_rate']).round(1)

fig_title = px.bar(
    surv, x='Title', y='survival_rate', text='survival_rate_pct',
    labels={'Title':'Título','survival_rate':'Taxa de Sobrevivência'},
    title='Taxa de Sobrevivência por Título (extraído de Name)'
)
fig_title.update_traces(texttemplate='%{text}%', textposition='outside')
fig_title.update_yaxes(range=[0,1], tickformat='.0%')
fig_title.show()
save_fig(fig_title, 'rate_by_title')

In [112]:
# --- Distribuição de idade por Título (violino) ---
df_age = df.dropna(subset=['Age']).copy()
fig_age = px.violin(
    df_age, x='Title', y='Age', box=True, points='all',
    title='Distribuição de Idade por Título',
    labels={'Title':'Título','Age':'Idade (anos)'}
)
fig_age.show()
save_fig(fig_age, 'violin_age_by_title')

In [113]:
# --- Qui-quadrado: Title × Survived ---
tab = pd.crosstab(df['Title'], df['Survived'])
st = chi_square_from_crosstab(tab)
print('--- Qui-quadrado (Title × Survived) ---')
print(tab)
print(f"χ² = {st['chi2']:.3f} | gl = {st['dof']} | p-valor = {st['p']:.3e}")
print(f"Cramér's V = {st['cramers_v']:.3f} | exp_min = {st['expected_min']:.2f}")

# --- Kruskal-Wallis: Age ~ Title ---
groups = [g['Age'].values for _, g in df_age.groupby('Title')]
H, p_kw = stats.kruskal(*groups)
print('\n--- Kruskal-Wallis (Age ~ Title) ---')
print(f"H = {H:.3f} | k = {len(groups)} grupos | p-valor = {p_kw:.3e}")


--- Qui-quadrado (Title × Survived) ---
Survived    0    1
Title             
Master     17   23
Miss       55  130
Mr        436   81
Mrs        26  100
Officer    13    5
Rare        2    3
χ² = 289.836 | gl = 5 | p-valor = 1.533e-60
Cramér's V = 0.570 | exp_min = 1.92

--- Kruskal-Wallis (Age ~ Title) ---
H = 195.819 | k = 6 grupos | p-valor = 2.227e-40


**Descrição**

* **Seleção e extração de `Title`**
  Parte do conjunto de treino e usa `extract_title` para obter o título formal do passageiro (Mr, Mrs, Miss, Master, Officer, Rare, …), normalizando e colapsando rótulos pouco frequentes.
  *Motivo:* o título sintetiza **gênero**, **faixa etária** e, em parte, **status social** — variáveis fortemente relacionadas ao desfecho no Titanic.

* **Taxa de sobrevivência por título**
  Agrega `Survived` por `Title` e plota a taxa (com rótulo em %) e o tamanho amostral.
  *Leitura esperada:* títulos femininos (Mrs/Miss) e infantis (Master) tendem a exibir taxas altas; `Mr` tipicamente baixa; grupos “Officer/Rare” variam conforme composição.

* **Distribuição de idade por título (violino)**
  Plota violinos com boxplot e pontos para comparar a distribuição contínua de `Age` entre os títulos.
  *Objetivo:* validar que os títulos realmente capturam diferenças de idade (ex.: Master mais jovem, Officer/rare mais velho), evitando assumir linearidade.

* **Associação categórica (qui-quadrado)**
  Monta a tabela `Title × Survived` e reporta χ², p-valor, gl, **Cramér’s V** e o menor valor esperado.
  *Uso:* confirmar estatisticamente que **o título está associado à sobrevivência** e medir a força desse efeito.

* **Diferenças de idade entre títulos (Kruskal-Wallis)**
  Teste não paramétrico para comparar **Age \~ Title** em múltiplos grupos.
  *Interpretação:* p pequeno indica que **ao menos um título** apresenta distribuição de idades distinta dos demais.

---

**Interpretação**

* **Taxas por título (gráficos)**

  * **Mrs \~79%**, **Miss \~70%**, **Master \~58%**, **Rare \~60%** → altos.
  * **Officer \~28%**, **Mr \~16%** → baixos.
  * Isso reflete o padrão “**women & children first**”: títulos femininos e infantis têm maior prioridade; “Mr” concentra a maior parte dos óbitos.

* **Associação com o desfecho (χ²)**

  * χ²(5)=**289,836**, **p ≈ 1,5×10⁻⁶⁰** → rejeita com grande folga a independência: **Title está fortemente associado à sobrevivência**.
  * **Cramér’s V = 0,570** → **efeito grande**.
  * **Atenção:** `exp_min = 1,92` indica células com contagem esperada muito baixa (especialmente em **Rare** e **Officer**). O resultado aponta forte associação mesmo assim, mas o teste χ² fica menos confiável; em relatórios, considere **agrupar categorias raras** (ex.: `Officer+Rare`) ou usar um **teste exato**.

* **Intuição com chances (exemplos)**

  * *Odds* (Mrs) = 100/26 ≈ **3,85** vs. *Odds* (Mr) = 81/436 ≈ **0,19** ⇒ **OR ≈ 20,7**.
    Mulheres casadas tiveram \~**21×** mais chances de sobreviver que “Mr”.
  * *Odds* (Miss) = 130/55 ≈ **2,36** ⇒ **OR(Miss vs Mr) ≈ 12,7**.

* **Idade por título (Kruskal–Wallis)**

  * H=**195,819**, **p ≈ 2,2×10⁻⁴⁰** → as **distribuições de idade diferem** entre títulos.
  * Violino confirma: **Master** é infantil, **Miss** jovem, **Mrs** mais velha que Miss, **Officer/Rare** tendem a idades mais altas, **Mr** concentrado em adultos.

* **Conclusão prática**

  * `Title` é uma **feature muito informativa**, pois codifica simultaneamente **gênero, idade** e parte do **status social**; explica boa parte da variação em `Survived`.
  * Para modelagem, use **one-hot/target encoding** e considere **interações com Sex e Pclass**.
  * Para testes formais/relatórios, agrupe categorias raras ou aplique teste exato para contagens pequenas.


# 8) Socioeconômico — Fare × Classe × Sobrevivência (Plots)

In [114]:
# 8) Socioeconômico — Fare × Classe × Sobrevivência (Plots)

df = train_only.copy()
df['Survived'] = df['Survived'].astype(int)
df['SurvivedLabel'] = np.where(df['Survived'].eq(1), 'Sobreviveu', 'Não sobreviveu')

# --- Boxplot: Fare × Pclass × Survived ---
fig = px.box(
    df.dropna(subset=['Fare','Pclass']),
    x='Pclass', y='Fare', color='SurvivedLabel',
    title='Distribuição de Fare por Classe e Sobrevivência',
    labels={'Pclass':'Classe', 'Fare':'Tarifa (Fare)', 'SurvivedLabel':'Desfecho'},
    points='outliers'
)
fig.update_yaxes(type='log')
fig.show()
save_fig(fig, 'box_fare_by_pclass_surv')

In [115]:
# --- Scatter: Age × Fare × Survived (contexto socioeconômico) ---
fig = px.scatter(
    df.dropna(subset=['Fare','Age']),
    x='Age', y='Fare', color='SurvivedLabel',
    title='Fare vs Age por Desfecho',
    labels={'Age':'Idade (anos)', 'Fare':'Tarifa (Fare)', 'SurvivedLabel':'Desfecho'},
    opacity=0.7
)
fig.update_yaxes(type='log')
fig.show()
save_fig(fig, 'scatter_age_fare_surv')


**Descrição**

* Selecionou apenas o `train`, criou o rótulo `SurvivedLabel` e:

  1. Fez **boxplots de `Fare` por `Pclass`**, colorindo pelo desfecho. O eixo-Y está em **log** para lidar com a forte assimetria de `Fare`.
  2. Fez um **dispersão `Age × Fare`**, colorindo pelo desfecho, também com `Fare` em log, para dar contexto socioeconômico (tarifa como proxy de classe/status).

**Como ler os gráficos / principais achados esperados**

* **Boxplot (`Fare × Pclass × Survived`)**

  * Em geral, **Pclass 1** tem `Fare` mais alto e **maior proporção de sobreviventes** (caixas/verdes mais altas).
  * **Pclass 3** concentra `Fare` baixo e **maioria de não sobreviventes**.
  * A escala log evidência a **cauda longa** de tarifas (outliers muito altos).
  * Interpretação: tarifa reflete **status socioeconômico** e **acesso aos botes** → forte relação com `Survived`.

* **Scatter (`Age × Fare × Survived`)**

  * Sobreviventes tendem a se concentrar em **faixas de `Fare` mais altas**, espalhados em diferentes idades.
  * Para `Fare` muito baixo, a densidade de **não sobreviventes** é maior.
  * O padrão por **idade** é fraco quando não estratificado por sexo/classe; o **eixo log** evita que o gráfico seja dominado por poucos pontos de `Fare` alto.

**Por que isso importa**

* `Fare` é um **forte preditor** e **proxy de Pclass**; usar ambos no modelo ajuda, mas verifique **colinearidade**.
* A relação não é puramente linear (efeitos de classe e política “women & children first”); análises estratificadas por **Sex** e **Pclass** ou um **modelo multivariado** dão leitura mais fiel.

---

**Interpretação**

* **Fare × Classe × Sobrevivência (boxplots, eixo log):**

  * A **Classe 1** tem tarifas muito mais altas e **maior proporção de sobreviventes**; a **Classe 3** concentra tarifas baixas e **prevalece não-sobrevivência**.
  * Dentro de **cada classe**, a **mediana de `Fare` dos sobreviventes é maior** que a dos não sobreviventes → `Fare` agrega informação **além** de `Pclass`.
  * Cauda longa: há **outliers** de `Fare` muito alto (especialmente na 1ª classe), majoritariamente sobreviventes.

* **Scatter `Age × Fare` por desfecho (eixo log em `Fare`):**

  * Há um **gradiente vertical claro**: acima de \~**40–60** de `Fare` predominam **sobreviventes**; abaixo de \~**10–15** predominam **não sobreviventes**.
  * **Idade** aparece **amplamente espalhada**; sem padrão forte isolado quando não estratificada por sexo/classe — reforça que **status/tarifa** explica mais do que idade aqui.

**Conclusão**

* `Fare` e `Pclass` funcionam como **proxies socioeconômicos** e estão **fortemente associados** à sobrevivência.
* Para modelagem:

  * usar **`log(Fare)`** (ou winsorização) devido à assimetria;
  * incluir **`Pclass` + `Fare`** (cuidar de colinearidade) e testar **interações** (ex.: `Sex×Pclass`, `Fare×Pclass`);
  * estratificações por **Sexo** evidenciam ainda mais o efeito (mulheres/pessoas com `Fare` alto tendem a sobreviver).


# 9) Conclusões e Próximos Passos

## Conclusões

* **Sexo é o fator isolado mais forte.** Mulheres tiveram \~74% de sobrevivência versus \~19% de homens; χ²(1)=260,7, p≪0,001, **Cramér’s V=0,54** (efeito grande). Evidência clara da política *women & children first*.
* **Status socioeconômico importa muito.** `Fare` (r\_point-biserial ≈ 0,26, p<0,001) e `Pclass` mostram forte associação: tarifas altas e 1ª classe concentram sobreviventes. Boxplots e dispersão (`Fare` em log) deixam esse gradiente visível.
* **Estrutura familiar tem efeito não linear.** Acompanhados (≥2) \~50,6% vs sozinhos 30,4% (χ² significativo; **V≈0,20**). Pico de sobrevivência em famílias **3–4**; queda em famílias grandes (≥5).
* **Idade contínua por si só não separa grupos**, mas faixas etárias revelam padrão: crianças 0–5 têm maior taxa (\~70%) e idosos têm a menor (\~23%). Mann–Whitney não significativo; χ² por faixas significativo com **V≈0,18**.
* **Cabine informada é proxy poderoso** (≈66,7% vs 30,0%; **V≈0,31**). *Deck* sugere tendências, mas o teste fica instável por células raras — precisa de agregação.
* **Título (extraído de `Name`) é extremamente informativo.** χ² muito significativo; **Cramér’s V≈0,57**. Títulos femininos e infantis (Mrs, Miss, Master) dominam as maiores taxas; `Mr` a menor. Kruskal–Wallis confirma que títulos capturam diferenças de idade.

**Resumo executivo:** Sobreviveram, sobretudo, **mulheres, crianças e passageiros de maior status** (1ª classe/tarifas altas/com cabine informada). Viajar **acompanhado** ajuda; grupos muito grandes e **idosos** sofrem desvantagem. O **Título** sintetiza (sexo + idade + status) e desponta como uma das melhores variáveis.

## Próximos passos

1. **Tratamento de faltantes**

   * `Age`: imputação estratificada por `Sex × Pclass × Title` (ou imputação por modelo, p.ex. RandomForest/LightGBM).
   * `Fare`: 1 valor nulo → mediana por `Pclass × Embarked`.
   * `Embarked`: imputar moda por classe/tarifa se houver.
2. **Refino de features**

   * `log(Fare)` ou winsorização para reduzir assimetria.
   * `FamilySize` categorizado (1; 2–4; ≥5) + `IsAlone`.
   * `Deck` agregado (ex.: {A/B/C}, {D/E/F}, {G/T/Other}, `Unknown` separado).
   * Interações candidatas: `Sex×Pclass`, `Sex×Title`, `Pclass×Fare`.
3. **Validação e modelagem**

   * *Baseline* com **Regressão Logística** (regularizada) e **árvores/gradient boosting** (RF/XGBoost/LightGBM).
   * **Validação estratificada** (k-fold), métricas **ROC-AUC** e **F1** da classe minoritária; *calibration curve* se necessário.
   * **SHAP**/importâncias para interpretação.
4. **Pipeline reproduzível**

   * `sklearn.Pipeline` com etapas de imputação, codificação (one-hot/target), escala (se aplicável), seleção de features.
   * *Data leakage guard*: imputações/encoders ajustados **apenas no treino** em cada *fold*.
5. **Robustez e documentação**

   * Exportar `reports/*.html` e `*.csv`, registrar ambiente (`pip freeze`), anotar limitações (ex.: esparsidade de `Deck`) e decisões.



# 10) Apêndice

* **Arquivos gerados**

  * `reports/eda_ydata_profiling.html`, `eda_sweetviz_overview.html`, `eda_sweetviz_target.html`.
  * `reports/describe_numericos.csv`, `describe_categoricos.csv`, `dic_dados_schema.csv`, `alvo_distribuicao_train.csv`.
* **Funções utilitárias**

  * `chi_square_from_crosstab` (χ², Cramér’s V, `expected_min`), `mannwhitney_stats` (U, p, AUC, rank-biserial), `collapse_rare`, `save_fig`, `gerar_relatorios_eda`.
* **Requisitos e ambiente**

  * Versões sugeridas: `pandas 1.5.x` (para ydata-profiling), `numpy 2.x` (com *workaround* do Sweetviz), `scipy`, `plotly`, `ydata-profiling`, `sweetviz`.
  * Lista de pacotes (`pip freeze`) e instruções de execução (incluindo “reiniciar kernel” após `%pip` quando necessário).
* **Checklist de qualidade**

  * [x] Sementes fixadas
  * [x] Sem vazamento entre treino e teste
  * [x] Premissas dos testes checadas (`expected_min`)
  * [x] Interpretações com tamanho de efeito
  * [x] Gráficos salvos com nomes normalizados
* **Limitações conhecidas**

  * `Deck` com forte esparsidade → testes com baixa potência; preferir agregação.
  * `Age` com \~20% faltantes → análises contínuas usam amostras menores.